# CSC411 Assignment 3

## Helper Functions for Part 2

In [13]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.image as mpimg
import os
import urllib
from copy import deepcopy
# Constants
# -----------------------------------------------------------------------------

In [14]:
# Functions used in multiple parts
# -----------------------------------------------------------------------------
def getNews(seed = 0):
    
    fakeFile = open("clean_fake.txt",'r')
    fakeNews = fakeFile.read().splitlines()
    fakeFile.close()
    realFile = open("clean_real.txt",'r')
    realNews = realFile.read().splitlines()
    realFile.close()
    
    np.random.shuffle(fakeNews)
    np.random.shuffle(realNews)
    
    return fakeNews, realNews

In [15]:
def dataSetSplit(fakeNews, realNews,seed = 0):
    
    trainingSet ={'real': realNews[:int(0.7*len(realNews))], 'fake':fakeNews[:int(0.7*len(fakeNews))]} 
    validationSet = {'real':realNews[int(0.7*len(realNews)):int(0.85*len(realNews))], 'fake':fakeNews[int(0.7*len(fakeNews)):int(0.85*len(fakeNews))]}
    testSet = {'real': realNews[int(0.85*len(realNews)):] , 'fake': fakeNews[int(0.85*len(fakeNews)):]}

    return trainingSet, validationSet, testSet

In [16]:
def count(trainingSet, validationSet, testSet):
    trainingCount = {'real':{}, 'fake':{}}
    validationCount = {'real':{}, 'fake':{}}
    testCount = {'real':{}, 'fake':{}}
    
    for rof in ['real','fake']:
        for news in trainingSet[rof]:
            for word in set(news.split(' ')):
                if not word in trainingCount[rof].keys():
                    trainingCount[rof][word] = 1
                else:
                    trainingCount[rof][word] += 1
        
        for news in validationSet[rof]:
            for word in set(news.split(' ')):
                if not word in validationCount[rof].keys():
                    validationCount[rof][word] = 1
                else:
                    validationCount[rof][word] += 1
        
        for news in testSet[rof]:
            for word in set(news.split(' ')):
                if not word in testCount[rof].keys():
                    testCount[rof][word] = 1
                else:
                    testCount[rof][word] += 1

    trainingRealCount = len(trainingSet['real'])
    trainingFakeCount = len(trainingSet['fake'])
    validationRealCount = len(validationSet['real'])
    validationFakeCount = len(validationSet['fake'])
    testRealCount = len(testSet['real'])
    testFakeCount = len(testSet['fake'])
    
    return trainingRealCount,trainingFakeCount,validationRealCount,validationFakeCount,testRealCount,testFakeCount, trainingCount, validationCount, testCount


In [17]:
def probability(trainingRealCount,trainingFakeCount, trainingCount, m, pHat):
    trainingProbability = {'real': {}, 'fake': {}}
    for word in trainingCount['real'].keys():
        trainingProbability['real'][word] = (trainingCount['real'][word]+m*pHat)/float(trainingRealCount+m)
    for word in trainingCount['fake'].keys():
        trainingProbability['fake'][word] = (trainingCount['fake'][word]+m*pHat)/float(trainingFakeCount+m)
    
    realProbability = float(trainingRealCount)/(trainingRealCount+trainingFakeCount)
    fakeProbability = float(trainingFakeCount)/(trainingRealCount+trainingFakeCount)
    
    return trainingProbability, realProbability, fakeProbability

In [18]:
def getAccuracy(trainingProbability,realProbability, fakeProbability, targetSet, m, pHat):
    accuracy = 0
    for rof in ['real', 'fake']:
        for numNews in range(len(targetSet[rof])):
            logProb = 0   
            for word in trainingProbability['real'].keys():
                if word in targetSet[rof][numNews].split(' '):
                    logProb += log(trainingProbability['real'][word])
                else:
                    logProb += log(1-trainingProbability['real'][word])
            for word in targetSet[rof][numNews].split(' '):
                if not word in trainingProbability['real'].keys():
                    logProb += log(pHat)
            logProb += log(realProbability)
            realProb = logProb

            logProb = 0 
            for word in trainingProbability['fake'].keys():
                if word in targetSet[rof][numNews].split(' '):
                    logProb += log(trainingProbability['fake'][word])
                else:
                    logProb += log(1-trainingProbability['fake'][word])
            for word in targetSet[rof][numNews].split(' '):
                if not word in trainingProbability['fake'].keys():
                    logProb += log(pHat)
            logProb += log(fakeProbability)
            fakeProb = logProb
            
            if rof == 'real':
                if realProb > fakeProb:
                    accuracy += 1
            else:
                if realProb < fakeProb:
                    accuracy += 1
                    
    return accuracy / (float(len(targetSet['fake']))+float(len(targetSet['real'])))


In [19]:
def generateWordBase(trainingSet):
    wordBase = []
    for rof in trainingSet.keys():
        for news in trainingSet[rof]:
            for word in news.split(' '):
                wordBase.append(word)
    return list(set(wordBase))

In [20]:
def getProbability(trainingProbability,realProbability, fakeProbability, wordBase, m, pHat):
    words = []
    realPs = []
    fakePs = []
    
    for testWord in wordBase:
        logProb = 0   
        for word in trainingProbability['real'].keys():
            if word == testWord:
                logProb += log(trainingProbability['real'][word])
            else:
                logProb += log(1-trainingProbability['real'][word])
        logProb += log(realProbability)
        realProb = logProb
        
        logProb = 0   
        for word in trainingProbability['fake'].keys():
            if word == testWord:
                logProb += log(trainingProbability['fake'][word])
            else:
                logProb += log(1-trainingProbability['fake'][word])
        logProb += log(fakeProbability)
        fakeProb = logProb
        
        words.append(testWord)
        realPs.append(realProb)
        fakePs.append(fakeProb)
        
    return words, realPs, fakePs


## Part 2

In [82]:
def part2(seed = 0):
    np.random.seed(seed)

    fakeNews, realNews = getNews()
    trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
    trainingRealCount,trainingFakeCount,validationRealCount, \
    validationFakeCount,testRealCount,testFakeCount, trainingCount, \
    validationCount, testCount = count(trainingSet, validationSet, testSet)
    
    m = 0.1
    pHat = 0.0001
    trainingProbability, realProbability, fakeProbability = \
    probability(trainingRealCount,trainingFakeCount, trainingCount, m,pHat)

    print getAccuracy(trainingProbability,realProbability, fakeProbability, validationSet, m, pHat)
    print getAccuracy(trainingProbability,realProbability, fakeProbability, testSet, m, pHat)
    print getAccuracy(trainingProbability,realProbability, fakeProbability, trainingSet, m, pHat)

## Part 3

In [ ]:
def part3Naive(seed = 0):
    np.random.seed(seed)
    
    trainingProbability, realProbability, fakeProbability = probability(trainingRealCount,trainingFakeCount, trainingCount, m, pHat)
    realWord = trainingProbability['real'].keys()
    fakeWord = trainingProbability['fake'].keys()
    realProbability = [trainingProbability['real'][prob] for prob in realWord]
    fakeProbability = [trainingProbability['fake'][prob] for prob in fakeWord]
    
    realResult = []
    for i in range(10):
        realResult.append(realWord.pop(realProbability.index(max(realProbability))))
        realProbability.remove(max(realProbability))
    
    fakeResult = []
    for i in range(10):
        fakeResult.append(fakeWord.pop(fakeProbability.index(max(fakeProbability))))
        fakeProbability.remove(max(fakeProbability))

In [173]:
def part3(seed = 0):
    np.random.seed(0)

    fakeNews, realNews = getNews()
    trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
    trainingRealCount,trainingFakeCount,validationRealCount, \
    validationFakeCount,testRealCount,testFakeCount, trainingCount, \
    validationCount, testCount = count(trainingSet, validationSet, testSet)

    m = 0.1
    pHat = 0.0001
    trainingProbability, realProbability, fakeProbability = \
    probability(trainingRealCount,trainingFakeCount, trainingCount, m,pHat)

    wordBase = generateWordBase(trainingSet)
    wordsOriginal, realPs, fakePs = getProbability(trainingProbability,realProbability, fakeProbability, wordBase, m, pHat)
    
    
    
    words = deepcopy(wordsOriginal)
    
    realResultMax = []
    for i in range(10):
        realResultMax.append(words.pop(realPs.index(max(realPs))))
        realPs.remove(max(realPs))

    words = deepcopy(wordsOriginal)
    fakeResultMax = []
    for i in range(10):
        fakeResultMax.append(words.pop(fakePs.index(max(fakePs))))
        fakePs.remove(max(fakePs))
    
    words = deepcopy(wordsOriginal)
    realResultMin = []
    for i in range(10):
        realResultMin.append(words.pop(realPs.index(min(realPs))))
        realPs.remove(min(realPs))
        
    words = deepcopy(wordsOriginal)
    fakeResultMin = []
    for i in range(10):
        fakeResultMin.append(words.pop(fakePs.index(min(fakePs))))
        fakePs.remove(min(fakePs))
        
    return realResultMax, fakeResultMax, realResultMin, fakeResultMin
    

In [174]:
realResultMax, fakeResultMax, realResultMin, fakeResultMin = part3()

In [179]:
fakeResultMax


['trump',
 'obstruction',
 'controversial',
 'aides',
 'marching',
 'stinks',
 'paris',
 'kidman',
 'condemns',
 'whack']

In [134]:
result = []
for i in range(10):
    result.append(fakeWord.pop(fakeProbability.index(max(fakeProbability))))
    fakeProbability.remove(max(fakeProbability))
result

['trump', 'to', 'the', 'donald', 'in', 'of', 'for', 'a', 'and', 'on']

## Helper Functions for Part 4

def generateWordBase(trainingSet):  #defined above
    wordBase = []
    for rof in trainingSet.keys():
        for news in trainingSet[rof]:
            for word in news.split(' '):
                wordBase.append(word)
    return list(set(wordBase))

In [62]:
def generateData(wordBase, targetSet):
    xy = np.zeros([len(wordBase)+2, (len(targetSet['real'])+len(targetSet['fake']))])
    n = 0
    for rof in ['real', 'fake']:
        for news in targetSet[rof]:
            for word in news:
                if word in wordBase:
                    xy[wordBase.index(word),n] = 1
            if rof == 'real':
                xy[-1,n] = 1
            n += 1
    xy[-2,:] = 1
    return xy

def sigmoidForward(y):
    return 1/(1+exp(-y))

def fcForward(x,W):
    return np.matmul(W.T,x)
    
def forward(x, W):
    L1 = fcForward(x,W)
    output = sigmoidForward(L1)
    return output

def sigmoidBackward(y,output):
    return output-y

def fcBackward(dy,x):
    dW = np.matmul(x,dy.T)
    return dW

def backward(y,output,x):
    dy = sigmoidBackward(y,output)
    dW = fcBackward(dy,x)
    return dW

def NLL(y, output):
    return -sum(y*log(output)+(1-y)*log((1-output)))

def backwardReg(y,output,x, W, lam):
    dy = sigmoidBackward(y,output)
    dW = fcBackward(dy,x)
    dWR = 2*lam*W
    return dW + dWR

def NLLReg(y,output, lam, W):
    return -sum(y*log(output)+(1-y)*log((1-output)))+ sum(lam*W)

def grad_descent(f, df, loss, x, y, init_t,ada_learning_rate = True, alpha=0.0001,max_iter=1000, EPS = 1e-5):
    
    prev_t = init_t-10*EPS
    t = init_t.copy()
    iter  = 0

    while norm(t - prev_t) >  EPS and iter < max_iter:
        
        #normal gradient descent
        prev_t = t.copy()
        t -= alpha*df(y, f(x,t), x)
        
        if iter % 5 == 0:
            print "Iter", iter
            currloss = loss(y, f(x,t))
            print currloss
        
        iter += 1
    
    return t

In [65]:
def grad_descent_regularization(f, df, loss, x, y, init_t,ada_learning_rate = True, \
                 tx=[], ty=[], figure_name = 'default',momentum = True, damping = 0.9, alpha=0.0001, \
                 max_iter=1000, EPS = 1e-5, zoom_in = 25):
    v = np.zeros((init_t.shape[0],init_t.shape[1])) #used for momentum
    lam = 0.0001 #######
    prev_t = init_t-10*EPS
    t = init_t.copy()
    
    currloss = loss(y, f(x,t), lam, t) # used for adaptive alpha
    curr_alpha = alpha # used for adaptive alpha
    
    iter  = 0
    num_iter = []
    performance_training = []
    performance_test = []
    
    while norm(t - prev_t) >  EPS and iter < max_iter:
        
        #normal gradient descent
        prev_t = t.copy()
        if momentum:
            v = damping*v+curr_alpha*df(y, f(x,t), x, t, lam)
            t -= v
        else:
            t -= curr_alpha*df(y, f(x,t), x, t, lam)
        
        if iter % 5 == 0:
            print "Iter", iter
            if currloss < loss(y, f(x,t), lam, t) and ada_learning_rate:
                curr_alpha = curr_alpha/2
            currloss = loss(y, f(x,t),lam, t)
            print currloss
        
        iter += 1
    
    return t

In [ ]:
def sigmoid(o):
    """
    sigmoid function
    """
    y = 1/(1+np.exp(-o))
    return y

def loss_fn(y, t):
    """
    loss function
    """
    return sum(-t*np.log(y) - (1-t)*np.log(1-y))


def f(x,t,W):
    """
    combined function of forward part
    """
    o = np.dot(x,W.T)
    y = sigmoid(o)
    return loss_fn(y, t)

def df(x,t,W):
    """
    combined function of back propagation part
    """
    o = np.dot(x,W.T)
    y = sigmoid(o)
    lossGradient = y-t
    combined_grad = np.dot(lossGradient.T, x)
    return combined_grad

In [64]:
def grad_descent(f, df, loss, x, y, init_t,ada_learning_rate = True, alpha=0.0001,max_iter=1000, EPS = 1e-5):
    
    prev_t = init_t-10*EPS
    t = init_t.copy()
    iter  = 0

    while norm(t - prev_t) >  EPS and iter < max_iter:
        
        #normal gradient descent
        prev_t = t.copy()
        t -= alpha*df(y, f(x,t), x)
        
        if iter % 5 == 0:
            print "Iter", iter
            currloss = loss(y, f(x,t))
            print currloss
        
        iter += 1
    
    return t

In [66]:
def performance(x, y, W, print_output = True,report = "test set"):
    output = forward(x,W)
    result = 0.
    print x.shape
    for i in range(x.shape[1]):
        if (output[0,i]>0.5 and y[i]==1):
            result+=1
        elif (output[0,i]<0.5 and y[i]==0):
            result += 1
            
    if print_output:
        print "Performance on "+report+ ": " +str(result)+"/"+str(x.shape[1])+"\n"
    
    return result/x.shape[1]

In [67]:
np.random.seed(0)
fakeNews, realNews = getNews()
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
wordBase = generateWordBase(trainingSet)

training_xy = generateData(wordBase, trainingSet)


In [70]:
W = np.random.normal(0.,0.,[len(wordBase)+1,1])

trainedW = grad_descent(f, df, NLL, training_xy[:-1,500:2500], training_xy[-1,500:2500], W,momentum = False, max_iter=500, alpha=0.0005,figure_name = "part4f1", tx=test_xy[:-1,:], ty=test_xy[-1,:])


Iter 0
1228.8643957063011
Iter 5
2013.106923007619
Iter 10
2215.1053398666145
Iter 15
1921.2931071288933
Iter 20
2048.7885317396767
Iter 25
1793.3365724795117
Iter 30
1886.2049073362864
Iter 35
1684.8504016208765
Iter 40
1754.2223109912147
Iter 45
1602.4953781112426
Iter 50
1661.2329619481407
Iter 55
1546.2112813004348
Iter 60
1601.4924825636106
Iter 65
1510.327191739977
Iter 70
1564.8045106131297
Iter 75
1488.0462497251092
Iter 80
1542.285393981389
Iter 85
1473.971003198493
Iter 90
1527.8822233674111
Iter 95
1464.581235748874
Iter 100
1517.9824042039795
Iter 105
1457.8274302185603
Iter 110
1510.5916596098816
Iter 115
1452.5890793327344
Iter 120
1504.6525643422128
Iter 125
1448.2689808956177
Iter 130
1499.610310318902
Iter 135
1444.547492505685
Iter 140
1495.1695145732817
Iter 145
1441.2484327932696
Iter 150
1491.1668785216725
Iter 155
1438.269588666164
Iter 160
1487.5066612608427
Iter 165
1435.547495376871
Iter 170
1484.128390266464
Iter 175
1433.0396645115513
Iter 180
1480.9906939748

In [73]:
performance(training_xy[:-1,500:2500], training_xy[-1,500:2500], trainedW, print_output = True,report = "training set")

(4791L, 1785L)
Performance on training set: 1181.0/1785



0.661624649859944

## Part 4


In [26]:
def part4(seed = 0):
    np.random.seed(seed)
    fakeNews, realNews = getNews()
    trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
    wordBase = generateWordBase(trainingSet)
    
    training_xy = generateData(wordBase, trainingSet)
    validation_xy = generateData(wordBase, validationSet)
    test_xy = generateData(wordBase, testSet)
    
    W = np.random.normal(0.,0.,[len(wordBase)+1,1])
    trainedW = grad_descent(forward, backward, NLL, training_xy[:-1,:], training_xy[-1,:], W,momentum = False, max_iter=2000, alpha=0.000,figure_name = "part4f1", tx=test_xy[:-1,:], ty=test_xy[-1,:])
    
    return training_xy, validation_xy, test_xy, trainedW

In [27]:
training_xy, validation_xy, test_xy, trainedW = part4()

KeyboardInterrupt: 

In [250]:
performance(training_xy[:-1,:], training_xy[-1,:], trainedW, print_output = True,report = "training set")

(5833L, 3266L)
Performance on training set: 2334.0/3266



0.7146356399265156

In [187]:
def part4R(seed = 0):
    np.random.seed(seed)
    fakeNews, realNews = getNews()
    trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
    wordBase = generateWordBase(trainingSet)
    
    training_xy = generateData(wordBase, trainingSet)
    validation_xy = generateData(wordBase, validationSet)
    test_xy = generateData(wordBase, testSet)
    
    W = np.random.normal(0.,0.,[len(wordBase)+1,1])
    trainedW = grad_descent_regularization(forward, backwardReg, NLLReg, training_xy[:-1,:], training_xy[-1,:], W,momentum = False, max_iter=2000, alpha=0.002,figure_name = "part4f1", tx=test_xy[:-1,:], ty=test_xy[-1,:])
    
    return training_xy, validation_xy, test_xy, trainedW

In [188]:
training_xy, validation_xy, test_xy, trainedW = part4R()

Iter 0
5498.930542057853
Iter 5
7276.3319723988625
Iter 10
3119.43739976914
Iter 15
2447.0727180082044
Iter 20
3305.596354627229
Iter 25
1370.7261286397
Iter 30
1367.203789451116
Iter 35
1364.316101765613
Iter 40
1361.8709182133466
Iter 45
1359.7881980600423
Iter 50
1358.0017840519292
Iter 55
1356.4573664064972
Iter 60
1355.1106292612901
Iter 65
1353.9255896539012
Iter 70
1352.8731445208089
Iter 75
1351.9298240736207
Iter 80
1351.0767390222804
Iter 85
1350.2987030120223
Iter 90
1349.5835089751452
Iter 95
1348.9213377452602
Iter 100
1348.3042783677326
Iter 105
1347.7259414269338
Iter 110
1347.181148956536
Iter 115
1346.6656868146092
Iter 120
1346.176107615317
Iter 125
1345.709574317856
Iter 130
1345.2637363387018
Iter 135
1344.8366315669043
Iter 140
1344.4266089359198
Iter 145
1344.0322672617426
Iter 150
1343.6524069228617
Iter 155
1343.285991660598
Iter 160
1342.9321183448242
Iter 165
1342.5899930036642
Iter 170
1342.2589117771024
Iter 175
1341.9382457409922
Iter 180
1341.6274287744918

Iter 1470
1324.9020827122845
Iter 1475
1324.8882566272323
Iter 1480
1324.8745168753533
Iter 1485
1324.8608626794255
Iter 1490
1324.8472932713278
Iter 1495
1324.8338078919098
Iter 1500
1324.8204057908627
Iter 1505
1324.8070862265874
Iter 1510
1324.793848466073
Iter 1515
1324.7806917847709
Iter 1520
1324.767615466474
Iter 1525
1324.7546188031981
Iter 1530
1324.7417010950637
Iter 1535
1324.7288616501821
Iter 1540
1324.7160997845403
Iter 1545
1324.703414821893
Iter 1550
1324.6908060936491
Iter 1555
1324.6782729387676
Iter 1560
1324.6658147036505
Iter 1565
1324.653430742039
Iter 1570
1324.6411204149115
Iter 1575
1324.6288830903825
Iter 1580
1324.616718143606
Iter 1585
1324.6046249566755
Iter 1590
1324.5926029185305
Iter 1595
1324.5806514248618
Iter 1600
1324.5687698780202
Iter 1605
1324.5569576869234
Iter 1610
1324.54521426697
Iter 1615
1324.533539039948
Iter 1620
1324.5219314339513
Iter 1625
1324.5103908832932
Iter 1630
1324.4989168284228
Iter 1635
1324.4875087158434
Iter 1640
1324.4761659

4790